### Parsing predicates from TPC-H queries

In [1]:
from tpch_query_predicates_parser import *
import json

In [2]:
# read workload queries from JSON file
def read_workload(workload_filepath):
    workload = []
    with open(workload_filepath) as f:
        line = f.readline()
        # read the queries from each line
        while line:
            workload.append(json.loads(line))
            line = f.readline()

    return workload

# Base directory containing the generated queries
workload_filepath = '../datagen/TPCH_workloads/TPCH_static_100_workload.json'

# Read the workload queries from file
workload = read_workload(workload_filepath)
print(len(workload))

2100


In [3]:
for template_num in range(1, 23):
    query_string = workload[template_num-1]['query_string']
    print(f"Query template: {template_num}")
    print("-" * 80)
    print(query_string)
    print("\nParsed predicates:")
    print("-" * 80)
    predicate_dict = parse_tpch_query(query_string, template_num)
    #print(predicate_dict)
    for table_name, predicates_list in predicate_dict.items():
        print(f"{table_name}:")
        for predicate in predicates_list:
            print("\t",predicate)
    print("\n\n")



Query template: 1
--------------------------------------------------------------------------------
select
	l_returnflag,
	l_linestatus,
	sum(l_quantity) as sum_qty,
	sum(l_extendedprice) as sum_base_price,
	sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
	sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
	avg(l_quantity) as avg_qty,
	avg(l_extendedprice) as avg_price,
	avg(l_discount) as avg_disc,
	count(*) as count_order
from
	lineitem
where
	l_shipdate <= DATEADD(dd, -112, CAST('1998-12-01' AS date))
group by
	l_returnflag,
	l_linestatus
order by
	l_returnflag,
	l_linestatus
;

Parsed predicates:
--------------------------------------------------------------------------------
lineitem:
	 {'column': 'l_shipdate', 'operator': '<=', 'value': "'1998-08-11'", 'join': False}



Query template: 2
--------------------------------------------------------------------------------

select
	s_acctbal,
	s_name,
	n_name,
	p_partkey,
	p_mfgr,
	s_address,
	s_phone,
	s_co